<a href="https://colab.research.google.com/github/qlegrand/Face-Detection/blob/master/ipynb/FaceRec_beta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install mxnet

In [0]:
!git clone https://gist.github.com/dc7e60aa487430ea704a8cb3f2c5d6a6.git /tmp/colab_util_repo
!mv /tmp/colab_util_repo/colab_util.py colab_util.py 
!rm -r /tmp/colab_util_repo
!pip install pydrive

Cloning into '/tmp/colab_util_repo'...
remote: Enumerating objects: 40, done.
remote: Total 40 (delta 0), reused 0 (delta 0), pack-reused 40
Unpacking objects: 100% (40/40), done.
    100% |████████████████████████████████| 993kB 26.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built pydrive


In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/Face\ Detection\ \(personal\ project\)
!pwd

/content/gdrive/My Drive/Colab Notebooks/Face Detection (personal project)
/content/gdrive/My Drive/Colab Notebooks/Face Detection (personal project)


In [0]:
import sys
sys.path.append('/content/gdrive/My Drive/Colab Notebooks/Face Detection (personal project)/ipynb/')

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
import keras
import os
import time
import mxnet as mx
import warnings

from keras import layers, activations, models, optimizers, utils, regularizers, initializers
from keras.engine.input_layer import Input
from keras.callbacks import Callback
from keras.preprocessing import image
import keras.backend as K
from sklearn.metrics import f1_score

from datetime import datetime as dt
from mxnet import recordio

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

if sys.version_info >= (3, 0, 0):
    import urllib.request as urllib # ugly but works
else:
    import urllib
    
import stl10_input
import FaceNetUtils as FNU
from colab_util import *

Using TensorFlow backend.


sys.version_info(major=3, minor=6, micro=7, releaselevel='final', serial=0)


In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
train_parameters = {
    'name'    :'FaceRec_siam',
    'seed'    :42,
    'base_lr' :1e-5,
    'max_lr'  : 5e-5,
    'lambda'  :0.0,
    'momentum':0.9,
    'batch_size':128,
    'epochs'  :2,
    'schedule':'triangle',
    'period'  :2000,
    'scaling' :0.9
    
}

run = dt.utcnow().strftime("%Y%m%d%H%M")

#MODEL_LOAD_PATH = './models/ResNet_run{}.h5'.format('201902270807')
MODEL_SAVE_PATH = './models/FRNet_run{}.h5'.format(run)
LOGS_FILE = "FRNet_run{}.csv".format(run)

In [0]:
DATA_DIR = './data/'
MODELS_DIR = './models/'
LOGS_DIR = './logs/'
DATA_PATH = './data/MS1M/'

gd_imgidx = './data/faces_emore/train.idx'
gd_imgrec = './data/faces_emore/train.rec'
path_imgidx = '/content/data/train.idx'
path_imgrec = '/content/data/train.rec'

In [0]:
"""MSM_refined_112x112 = 'https://redcrossstorage.blob.core.windows.net/datasets/faces_ms1m-refine-v2_112x112.zip'
VGG2_112x112 = 'https://redcrossstorage.blob.core.windows.net/datasets/faces_vgg2_112x112.zip'
Emore = 'https://redcrossstorage.blob.core.windows.net/datasets/faces_emore.zip'
FNU.download_and_extract(MSM_refined_112x112,DATA_DIR,task='e', force=True)"""

"MSM_refined_112x112 = 'https://redcrossstorage.blob.core.windows.net/datasets/faces_ms1m-refine-v2_112x112.zip'\nVGG2_112x112 = 'https://redcrossstorage.blob.core.windows.net/datasets/faces_vgg2_112x112.zip'\nEmore = 'https://redcrossstorage.blob.core.windows.net/datasets/faces_emore.zip'\nFNU.download_and_extract(MSM_refined_112x112,DATA_DIR,task='e', force=True)"

In [0]:
class SiamSequenceGenerator(utils.Sequence):
    def __init__(self,images_files_structure, batch_size, path_idx, path_rec, skew=0.05):
        self.images_files_structure = images_files_structure
        self.dir_list = np.array(list(images_files_structure.keys()))
        self.batch_size = batch_size
        self.skew = skew
        self.path_idx = path_idx
        self.path_rec = path_rec
        self.imgrec = recordio.MXIndexedRecordIO(path_idx, path_rec, 'r')
        chosen_persons = []
        chosen_pictures = []
        for key,val in images_files_structure.items():
            if len(val) != 0:
                for v in val:    
                    chosen_persons.append(key)
                    chosen_pictures.append(v)
            else:
                print('Person {} has no picture'.format(key))
                             
        self.persons_list = np.array(chosen_persons)
        self.pictures_list = np.array(chosen_pictures)
        self.num_img = len(chosen_pictures)
        self.length = int(np.ceil(self.num_img/float(self.batch_size)))
        self.on_epoch_begin()
        
    def __len__(self):
        return self.length
    
    def on_epoch_begin(self):
        skew_count = 0
        init_permutations = np.random.permutation(len(self.persons_list))
        chosen_persons_1 = self.persons_list[init_permutations]
        self.chosen_pictures_1 = self.pictures_list[init_permutations]
        self.chosen_pictures_2 = []
        self.labels = []
        
        for person in chosen_persons_1:       
            if skew_count < np.ceil(self.skew * len(self.pictures_list)):
                chosen_person_2 = person
                label = 1
                skew_count += 1
            else:
                label = 0
                while True:
                    chosen_person_2 = chosen_persons_1[np.random.choice(len(chosen_persons_1))]
                    if len(self.images_files_structure[chosen_person_2]) != 0:
                        break
                
            self.chosen_pictures_2.append(self.images_files_structure[chosen_person_2][np.random.choice(len(self.images_files_structure[chosen_person_2]))])
            self.labels.append(label)
            
        permutations = np.random.permutation(len(self.persons_list))
        self.chosen_pictures_1 = self.chosen_pictures_1[permutations]
        self.chosen_pictures_2 = np.array(self.chosen_pictures_2)[permutations]
        self.labels = np.array(self.labels)[permutations]
     
    def __getitem__(self,idx):
        batch_idx_1 = self.chosen_pictures_1[idx * self.batch_size:(idx + 1) * self.batch_size] 
        batch_idx_2 = self.chosen_pictures_2[idx * self.batch_size:(idx + 1) * self.batch_size] 
        batch_y = self.labels[idx * self.batch_size:(idx + 1) * self.batch_size]
        img_1 = np.reshape([mx.image.imdecode(recordio.unpack(self.imgrec.read_idx(idx_1))[1]).asnumpy() for idx_1 in batch_idx_1],(-1,112,112,3))
        img_2 = np.reshape([mx.image.imdecode(recordio.unpack(self.imgrec.read_idx(idx_2))[1]).asnumpy() for idx_2 in batch_idx_2],(-1,112,112,3))
        
        return [img_1/255 ,img_2/255] , batch_y.astype(float)
      
class SaveModelandReduceLR(Callback):
    def __init__(self,measure,val_generator,path,period,reduce_lr=True,factor=10,patience=5,min_lr=1e-8):
        self.measure = measure
        self.period = period
        self.path = path
        self.generator = val_generator
        self.reduce_lr = reduce_lr
        self.patience = patience
        self.min_lr = min_lr
        self.factor = factor
                   
    def on_train_begin(self, logs={}):
        self.best_measure = 0.0
        self.increment = 1
        eval_measure_stack = [] 
        self.improvement_counter = 1
        for i in range(self.generator.length):
            batch_x, targ = self.generator.__getitem__(i)
            predict = (np.asarray(self.model.predict(batch_x)))
            eval_measure_stack.append(self.measure(np.squeeze(targ), np.squeeze(predict.round())))
        eval_measure = np.mean(eval_measure_stack)
        print('\nval {}: {}'.format(self.measure.__name__, eval_measure))
        if eval_measure >= self.best_measure:
            self.best_measure = eval_measure
        
    def on_batch_end(self, batch, logs={}):
        eval_measure_stack = [] 
        if self.increment % self.period == 0:
            for i in range(self.generator.length):
                batch_x, targ = self.generator.__getitem__(i)
                predict = (np.asarray(self.model.predict(batch_x)))
                eval_measure_stack.append(self.measure(np.squeeze(targ), np.squeeze(predict.round())))
            eval_measure = np.mean(eval_measure_stack)
            print('\nval {}: {}'.format(self.measure.__name__, eval_measure))
            
            if eval_measure > self.best_measure:
                self.improvement_counter = 1
                self.best_measure = eval_measure
                self.model.save(self.path)
                print('Model saved')
            elif self.reduce_lr:
                if self.improvement_counter >= self.patience:
                    self.improvement_counter = 1
                    reduced_lr = max(K.get_value(self.model.optimizer.lr)/self.factor,self.min_lr)
                    if K.get_value(self.model.optimizer.lr) >= self.min_lr:
                        print("LR reduced on plateau to {}".format(reduced_lr))
                    K.set_value(self.model.optimizer.lr, reduced_lr)
                    
                else:
                    self.improvement_counter += 1
                    
        self.increment += 1
        
    def on_epoch_end(self, epoch, logs={}):
        eval_measure_stack = [] 
        self.increment = 1
        for i in range(self.generator.length):
            batch_x, targ = self.generator.__getitem__(i)
            predict = (np.asarray(self.model.predict(batch_x)))
            eval_measure_stack.append(self.measure(np.squeeze(targ), np.squeeze(predict.round())))
        eval_measure = np.mean(eval_measure_stack)
        print('\nval {}: {}'.format(self.measure.__name__, eval_measure))
        if eval_measure >= self.best_measure:
            self.best_measure = eval_measure
            self.model.save(self.path)
            print('Model saved')
            
class LRSearch(Callback):
    def __init__(self, measure, val_generator,min_lr=1.0e-8, max_lr=3.0, method="exp", train_iterations = 10000, val_points = 100, data_skew = 0.05):
        self.min_lr = min_lr
        self.max_lr = max_lr
        self.method = method
        self.skew = data_skew
        self.total_steps = train_iterations
        self.validation_steps = val_points
        self.generator = val_generator
        self.measure = measure
        warnings.filterwarnings("ignore")
        
    def on_train_begin(self, logs={}):
        if self.method == "exp":
            self.base = np.power((self.max_lr/self.min_lr),1/(self.total_steps -1))
        else:
            self.base = (self.max_lr - self.min_lr)/(self.total_steps -1)
        self.lr_sched = self.min_lr
        self.losses = []
        self.accuracies = []
        self.plot_x = []
        self.plot_x_val = []
        self.val_measure = []
        self.iteration_count = 0
        self.val_checkpoints = round(self.total_steps/self.validation_steps)
        return
    
    def on_batch_begin(self, batch, logs={}):
        K.set_value(self.model.optimizer.lr, self.lr_sched)
        return
        
    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs['loss'])
        self.accuracies.append((logs['acc']-1+self.skew)/self.skew)
        self.plot_x.append(self.lr_sched)
        
        if self.iteration_count % self.val_checkpoints == 0:
            eval_measure_stack = [] 
            for i in range(self.generator.length):
                batch_x, targ = self.generator.__getitem__(i)
                predict = (np.asarray(self.model.predict(batch_x)))
                eval_measure_stack.append(self.measure(np.squeeze(targ), np.squeeze(predict.round())))
            self.val_measure.append(np.mean(eval_measure_stack))
            self.plot_x_val.append(self.lr_sched)
        
        self.iteration_count +=1
        if self.iteration_count >= self.total_steps:
            self.model.stop_training = True
        
        if self.method=="exp":
            self.lr_sched *= self.base
        else:
            self.lr_sched += self.base
        return
      
    def on_train_end(self, epoch, logs={}):
        alpha = 0.95
        plt.figure(figsize = (10, 6))
        plt.plot(self.plot_x)
        plt.xlabel("steps")
        plt.ylabel("LR")
        plt.show()
      
        plot_y_measure = self.__ewma_vectorized(self.val_measure, alpha)
        plt.figure(figsize = (10, 6))
        plt.xlabel("LR")
        plt.ylabel("Validation {}".format(self.measure.__name__))
        plt.plot(self.plot_x_val, plot_y_measure)
        plt.savefig('./figures/val_{}.png'.format(self.measure.__name__))
        plt.show()
        
        plt.figure(figsize = (10, 6))
        plt.xscale("log")
        plt.xlabel("LOG LR")
        plt.ylabel("Validation {}".format(self.measure.__name__))
        plt.plot(self.plot_x_val, plot_y_measure)
        plt.savefig('./figures/log_val_{}.png'.format(self.measure.__name__))
        plt.show()
        
        plot_y_loss = self.__ewma_vectorized(self.losses, alpha)
        plt.figure(figsize = (10, 6))
        plt.xlabel("LR")
        plt.ylabel("Train Loss")
        plt.plot(self.plot_x, plot_y_loss)
        plt.savefig('./figures/train_losses.png')
        plt.show()
        
        plt.figure(figsize = (10, 6))
        plt.xscale("log")
        plt.xlabel("LOG LR")
        plt.ylabel("Train Loss")
        plt.plot(self.plot_x, plot_y_loss)
        plt.savefig('./figures/log_train_losses.png')
        plt.show()
        
        plot_y_acc = self.__ewma_vectorized(self.accuracies, alpha)
        plt.figure(figsize = (10, 6))
        plt.xlabel("LR")
        plt.ylabel("Train Accuracies")
        plt.plot(self.plot_x, plot_y_acc)
        plt.savefig('./figures/train_accuracies.png')
        plt.show()
        
        plt.figure(figsize = (10, 6))
        plt.xscale("log")
        plt.xlabel("LOG LR")
        plt.ylabel("Train Accuracies")
        plt.plot(self.plot_x, plot_y_acc)
        plt.savefig('./figures/log_train_accuracies.png')
        plt.show()
    
    def __ewma_vectorized(self, data, alpha):
        alpha_ = 1-alpha
        D = pd.DataFrame(data)
        M = D.ewm(alpha=alpha_).mean()
        out = M.values
        return out
      
class LRScheduler(Callback):
    def __init__(self, max_lr, scale_factor=1.0, period=1000, schedule="triangle", plot_schedule=True): 
        self.scale_factor = scale_factor
        self.max_lr = max_lr
        self.steps_in_period =  period
        self.schedule = schedule
        self.plot_schedule = plot_schedule
        
    def on_train_begin(self, logs={}):
        if self.params['steps'] != None:
            self.steps_in_epoch = self.params['steps']
        else:
            self.steps_in_epoch = np.ceil(self.params['samples'] / self.params['batch_size'])
        self.step = 0
        self.nb_steps = 0
        self.amplitude_lr = 0
        self.slope_lr = 0
        self.min_lr = K.get_value(self.model.optimizer.lr)
        self.saved_lr = K.get_value(self.model.optimizer.lr)
        self.second_period = 0
        
        if self.schedule == "triangle":
            self.amplitude_lr = (self.max_lr - self.min_lr)
        else:
            self.frequency = np.pi/self.steps_in_period
            self.amplitude_lr = (self.max_lr - self.min_lr)/2
            
        self.lr_stack = []
        return
    
    def on_batch_begin(self, batch, logs={}):
        if self.schedule == "triangle":
            x = abs(- self.amplitude_lr + 2 * self.amplitude_lr/self.steps_in_period * self.step)
            learning_rate = self.min_lr + max(0, self.amplitude_lr - x)
        else:
            learning_rate = self.amplitude_lr * np.cos(self.step * self.frequency) + self.amplitude_lr + self.min_lr
        K.set_value(self.model.optimizer.lr, learning_rate)
        self.lr_stack.append(learning_rate)
        self.nb_steps+=1
        self.step+=1
        return
    
    def on_batch_end(self, batch, logs={}):
        batch_ = batch + 1
        if batch_ % self.steps_in_period == 0:     
            self.step = 0
            self.amplitude_lr *= self.scale_factor
            if self.second_period < 2: 
                self.plot_lr_schedule()
                self.second_period += 1
        return
    
    def on_train_end(self, epoch, logs={}):    
        K.set_value(self.model.optimizer.lr, self.saved_lr)
        if self.plot_schedule: self.plot_lr_schedule()
        return

    def plot_lr_schedule(self):
        plt.figure(figsize = (5, 3))
        plt.plot(np.arange(self.nb_steps), self.lr_stack)
        plt.xlabel("#steps")
        plt.ylabel("lr schedule")
        plt.show()

In [0]:
kilobytes = 1024
megabytes = kilobytes * 1024
gigabytes = megabytes * 1024
chunksize = int(1.1 * gigabytes)                   # default: roughly a floppy
readsize = kilobytes

def split(fromfile, todir, chunksize=chunksize): 
    if not os.path.exists(todir):                  # caller handles errors
        os.mkdir(todir)                            # make dir, read/write parts
    else:
        for fname in os.listdir(todir):            # delete any existing files
            os.remove(os.path.join(todir, fname))           
    partnum = 0
    input = open(fromfile, 'rb')                   # use binary mode on Windows
    while 1:                                       # eof=empty string from read
        chunk = input.read(chunksize)              # get next part <= chunksize
        if not chunk: break
        partnum  = partnum+1
        filename = os.path.join(todir, ('part%04d' % partnum))
        fileobj  = open(filename, 'wb')
        fileobj.write(chunk)
        fileobj.close()                            # or simply open(  ).write(  )
    input.close()
    assert partnum <= 9999                         # join sort fails if 5 digits
    return partnum
  
def join(fromdir, tofile,readsize=megabytes):
    output = open(tofile, 'wb')
    parts  = os.listdir(fromdir)
    parts.sort()
    for filename in parts:
        filepath = os.path.join(fromdir, filename)
        fileobj  = open(filepath, 'rb')
        while 1:
            filebytes = fileobj.read(readsize)
            if not filebytes: break
            output.write(filebytes)
        fileobj.close()
    output.close()
    
def PrepareDataset(joinfunc): 
    idxfile = 'train.idx'
    recfile = 'train.rec'
    fromIDXdir = '/Colab Notebooks/Face Detection (personal project)/data/faces_emore/'
    fromRECpartsdir = '/Colab Notebooks/Face Detection (personal project)/data/MS1MSplit/'
    fromRECpartsreldir = './data/MS1MSplit/'
    toRECpartsdir = '/content/MS1MParts/'
    todatadir = '/content/data/'
    
    drive_handler = GoogleDriveHandler()
  
    if not os.path.exists(toRECpartsdir):
        os.mkdir(toRECpartsdir)
        parts  = os.listdir(fromRECpartsreldir)
        for filename in parts: 
            drive_handler.download(os.path.join(toRECpartsdir,filename), target_path=os.path.join(fromRECpartsdir,filename))
            print('{} moved'.format(filename))
       
    if not os.path.exists(todatadir):
        os.mkdir(todatadir)
        drive_handler.download(os.path.join(todatadir,idxfile), target_path=os.path.join(fromIDXdir,idxfile))
        print('{} moved'.format(idxfile))
        print('\nAll files moved')
        joinfunc(toRECpartsdir,os.path.join(todatadir,recfile))
        print('\nDataset Ready!')
        
def LoadModel(model,path):
    files  = os.listdir(path)
    most_recent = 0
    for file in files:
        date = int(file[-15:-3])
        if date > most_recent:
            most_recent = date
            mr_file = file
    print('Most recent checkpoint {}/{}/{} {}:{} ({})'.format(str(most_recent)[6:8],str(most_recent)[4:6],str(most_recent)[:4],str(most_recent)[8:10],str(most_recent)[10:],mr_file))
    model_to_load = os.path.join(path,mr_file)
    model.load_weights(model_to_load, by_name=True)
    print('Model {} loaded!'.format(mr_file))
    
def ExtractRECStructure(rec_file,dataset_structure=DATA_DIR):
    structure_log = os.path.join(dataset_structure,'dataset_structure.csv')

    if not os.path.exists(structure_log):
        structure = {} 
        max_ = 5908395
        imrec = recordio.MXRecordIO(rec_file, 'r')
        for i in range(max_):
            sys.stdout.write('\rReading structure {:.4f}%'.format(float(i)/float(max_) * 100.0))
            sys.stdout.flush()
            header, _ =  recordio.unpack(imrec.read())
            try:
                structure[int(header.label)].append(header.id)
            except KeyError:
                structure[int(header.label)] = [header.id]
            if i % 500000 == 0 : 
                df = pd.DataFrame.from_dict(structure, orient='index')
                df.to_csv(structure_log)
                print('\nStructure saved')
        df = pd.DataFrame.from_dict(structure, orient='index')
        df.to_csv(structure_log)
        print('\nStructure saved')
    else:
        df = pd.read_csv(structure_log, index_col=0)
        tmp_dict = df.to_dict('index')
        structure = {u: np.array([int(x) for w,x in v.items() if pd.notna(x)]) for u,v in tmp_dict.items()}
        
    print('\nStructure extracted')
    return structure
  
def SplitRECStructure(structure,val_length=100):
    num_persons = len(list(structure.keys()))
    
    split = num_persons - min(val_length, int(num_persons*0.1))

    train_structure = {key:val for key,val in structure.items() if key <= split}
    val_structure = {key:val for key,val in structure.items() if key > split} 
    
    return train_structure, val_structure

In [0]:
"""to_folder = './data/MS1MSplit'
split(gd_imgrec,to_folder)"""

"to_folder = './data/MS1MSplit'\nsplit(gd_imgrec,to_folder)"

In [0]:
PrepareDataset(join)

part0001 moved
part0002 moved
part0003 moved
part0004 moved
part0005 moved
part0006 moved
part0007 moved
part0008 moved
part0009 moved
part0010 moved
part0011 moved
part0012 moved
part0013 moved
part0014 moved
part0015 moved
train.idx moved

All files moved

Dataset Ready!


In [0]:
structure = ExtractRECStructure(gd_imgrec)


Structure extracted


In [0]:
train_structure, val_structure = SplitRECStructure(structure)

In [0]:
channels = [
    [64, 64, 256],
    [128, 128, 512],
    [256, 256, 1024],
    [512, 512, 2048]
]

input_ = Input(shape=(112,112,3))
x = layers.Conv2D(64,(7,7),kernel_initializer=initializers.he_normal(seed=train_parameters['seed']),strides=2, padding='same',kernel_regularizer=regularizers.l2(train_parameters['lambda']), name="block_0_cv_0")(input_)
x = layers.BatchNormalization(name="block_0_bn_0")(x)
x = layers.Activation('relu', name="block_0_relu_0")(x)
x = layers.MaxPooling2D(pool_size=(2, 2), strides=2, name="block_0_mp_0")(x)

x = FNU.conv_block(x, channels[0], train_parameters['lambda'], 1, strides=1, seed=train_parameters['seed'])
x = FNU.id_conv_block(x, channels[0], train_parameters['lambda'], 2, seed=train_parameters['seed'])
x = FNU.id_conv_block(x, channels[0], train_parameters['lambda'], 3, seed=train_parameters['seed'])
x = FNU.conv_block(x, channels[1], train_parameters['lambda'], 4, seed=train_parameters['seed'])
x = FNU.id_conv_block(x, channels[1], train_parameters['lambda'], 5, seed=train_parameters['seed'])
x = FNU.id_conv_block(x, channels[1], train_parameters['lambda'], 6, seed=train_parameters['seed'])
x = FNU.id_conv_block(x, channels[1], train_parameters['lambda'], 7, seed=train_parameters['seed'])
x = FNU.conv_block(x, channels[2], train_parameters['lambda'], 8, seed=train_parameters['seed'])
x = FNU.id_conv_block(x, channels[2], train_parameters['lambda'], 9, seed=train_parameters['seed'])
x = FNU.id_conv_block(x, channels[2], train_parameters['lambda'], 10, seed=train_parameters['seed'])
x = FNU.id_conv_block(x, channels[2], train_parameters['lambda'], 11, seed=train_parameters['seed'])
x = FNU.id_conv_block(x, channels[2], train_parameters['lambda'], 12, seed=train_parameters['seed'])
x = FNU.id_conv_block(x, channels[2], train_parameters['lambda'], 13, seed=train_parameters['seed'])
x = FNU.conv_block(x, channels[3], train_parameters['lambda'], 14, seed=train_parameters['seed'])
x = FNU.id_conv_block(x, channels[3], train_parameters['lambda'], 15, seed=train_parameters['seed'])
x = FNU.id_conv_block(x, channels[3], train_parameters['lambda'], 16, seed=train_parameters['seed'])
x = layers.AveragePooling2D(pool_size=(3, 3), strides=1)(x)#different from model to adapt to STL-10 // pool_size=(7, 7)
x = layers.Flatten()(x)

x = layers.Dense(128, activation = 'linear', name = "features_dense_0")(x)
x = layers.BatchNormalization(name = 'features_bn_0')(x)
output_ = layers.Activation('relu', name = 'features_relu_0')(x)

features_model = models.Model(inputs=input_, outputs=output_)

LoadModel(features_model,MODELS_DIR)

features_model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Most recent checkpoint 01/05/2019 07:18 (FRNet_run201905010718.h5)
Model FRNet_run201905010718.h5 loaded!
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 112, 112, 3)  0                                            
__________________________________________________________________________________________________
block_0_cv_0 (Conv2D)           (None, 56, 56, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
block_0_bn_0 (BatchNormalizatio (None, 56, 56, 64)   256         block_0_cv_0[0][0]               
__________________________________________________________________________________________________
block_0_relu_0 (Activation)   

In [0]:
input_a = Input(shape=(112,112,3), name='input_a')
input_b = Input(shape=(112,112,3), name='input_b')

features_a = features_model(input_a)
features_b = features_model(input_b)

combined_features = layers.concatenate([features_a, features_b], name = 'combined_features_concatenate_0')

combined_features = layers.Dense(256, activation = 'linear', name = 'extended_combined_features_dense_0')(combined_features)
combined_features = layers.BatchNormalization(name = 'extended_combined_features_bn_0')(combined_features)
combined_features = layers.Activation('relu', name = 'extended_combined_features_relu_0')(combined_features)

combined_features = layers.Dense(256, activation = 'linear', name = 'extended_combined_features_dense_1')(combined_features)
combined_features = layers.BatchNormalization(name = 'extended_combined_features_bn_1')(combined_features)
combined_features = layers.Activation('relu', name = 'extended_combined_features_relu_1')(combined_features)

combined_features = layers.Dense(128, activation = 'linear', name = 'extended_combined_features_dense_2')(combined_features)
combined_features = layers.BatchNormalization(name = 'extended_combined_features_bn_2')(combined_features)
combined_features = layers.Activation('relu', name = 'extended_combined_features_relu_')(combined_features)

combined_features = layers.Dense(128, activation = 'linear', name = 'extended_combined_features_dense_3')(combined_features)
combined_features = layers.BatchNormalization(name = 'extended_combined_features_bn_3')(combined_features)
combined_features = layers.Activation('relu', name = 'extended_combined_features_relu_3')(combined_features)

combined_features = layers.Dense(64, activation = 'linear', name = 'extended_combined_features_dense_4')(combined_features)
combined_features = layers.BatchNormalization(name = 'extended_combined_features_bn_4')(combined_features)
combined_features = layers.Activation('relu', name = 'extended_combined_features_relu_4')(combined_features)

combined_features = layers.Dense(64, activation = 'linear', name = 'extended_combined_features_dense_5')(combined_features)
combined_features = layers.BatchNormalization(name = 'extended_combined_features_bn_5')(combined_features)
combined_features = layers.Activation('relu', name = 'extended_combined_features_relu_5')(combined_features)

model_output = layers.Dense(1, activation = 'sigmoid', name = 'extended_fn_output_0')(combined_features)

similarity_model = models.Model(inputs = [input_a, input_b], outputs = model_output, name = 'Similarity_Model')

LoadModel(similarity_model,MODELS_DIR)

similarity_model.summary()

Most recent checkpoint 01/05/2019 07:18 (FRNet_run201905010718.h5)
Model FRNet_run201905010718.h5 loaded!
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_a (InputLayer)            (None, 112, 112, 3)  0                                            
__________________________________________________________________________________________________
input_b (InputLayer)            (None, 112, 112, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 128)          24636928    input_a[0][0]                    
                                                                 input_b[0][0]                    
__________________________________________________________________________________________________
com

In [0]:
for index in range(175):
    features_model.get_layer(index=index).trainable = False
features_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 112, 112, 3)  0                                            
__________________________________________________________________________________________________
block_0_cv_0 (Conv2D)           (None, 56, 56, 64)   9472        input_1[0][0]                    
__________________________________________________________________________________________________
block_0_bn_0 (BatchNormalizatio (None, 56, 56, 64)   256         block_0_cv_0[0][0]               
__________________________________________________________________________________________________
block_0_relu_0 (Activation)     (None, 56, 56, 64)   0           block_0_bn_0[0][0]               
__________________________________________________________________________________________________
block_0_mp

In [0]:
optimizer = optimizers.SGD(lr=train_parameters['base_lr'], momentum=train_parameters['momentum'],nesterov=True)

similarity_model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy'])
similarity_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_a (InputLayer)            (None, 112, 112, 3)  0                                            
__________________________________________________________________________________________________
input_b (InputLayer)            (None, 112, 112, 3)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 128)          24636928    input_a[0][0]                    
                                                                 input_b[0][0]                    
__________________________________________________________________________________________________
combined_features_concatenate_0 (None, 256)          0           model_1[1][0]                    
          

In [0]:
train_generator = SiamSequenceGenerator(train_structure,train_parameters['batch_size'],path_imgidx,path_imgrec)
val_generator = SiamSequenceGenerator(val_structure,128,path_imgidx,path_imgrec)
train_s_p_e = train_generator.length
val_s_p_e = val_generator.length

In [0]:
callbacks_list = [LRSearch(f1_score,val_generator)]
similarity_model.fit_generator(train_generator, steps_per_epoch=train_s_p_e,epochs=train_parameters['epochs'], validation_data=val_generator, validation_steps=val_s_p_e, callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Epoch 1/2
 2282/42920 [>.............................] - ETA: 7:06:03 - loss: 0.2020 - acc: 0.9498

In [0]:
"""callbacks_list = [SaveModelandReduceLR(f1_score,val_generator,MODEL_SAVE_PATH, 1000,reduce_lr = False), LRScheduler(train_parameters['max_lr'], period =train_parameters['period'])]
similarity_model.fit_generator(train_generator, steps_per_epoch=train_s_p_e,epochs=train_parameters['epochs'], validation_data=val_generator, validation_steps=val_s_p_e, callbacks=callbacks_list)"""

In [0]:
"""for index in range(175):
    features_model.get_layer(index=index).trainable = True
similarity_model.save(MODEL_SAVE_PATH)"""